# import libs

In [ ]:
import os
import socket
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
from FCNN import TwoInputFCNN as FCNN


# import data

In [ ]:
# paths pre-setting
if 'gput' in socket.gethostname() or 'hpc' in socket.gethostname():
    label_root = '/home/sxy841/ERIE/silicone/output'
    model_root = '/home/sxy841/ERIE/silicone/Track-Shuyuan-2023-06-13/videos'
else:
    label_root = '/home/ysy/ERIE/single_psm_manipulation_dataset/output'
    model_root = '/home/ysy/ERIE/videos'

model_name = 'DLC_resnet50_TrackJun13shuffle1_50000'



In [ ]:
# pre setting training sets
training_sets = ['C_M1_T1_1', 'C_M1_T1_3', 'C_M1_T1_5', 'C_M1_T1_7', 
                 'R2_M1_T1_1', 'R2_M1_T1_3', 'R2_M1_T1_5', 'R2_M1_T1_7', 
                 'L2_M1_T1_1', 'L2_M1_T1_3', 'L2_M1_T1_5', 'L2_M1_T1_7', 
                 'Z2_M1_T1_1', 'Z2_M1_T1_3', 'Z2_M1_T1_5']

# initial arrays
X_train_L = np.zeros((0, 16))
X_train_R = np.zeros((0, 16))
y_train = np.zeros((0, 3))

for set in tqdm(training_sets):
    # load from files
    labels = np.genfromtxt(os.path.join(
        label_root, set, 'labels_30hz.txt'), delimiter=',')
    coordinates_L = pd.read_hdf(os.path.join(
        model_root, f'{set}_L_h264{model_name}.h5'))
    coordinates_R = pd.read_hdf(os.path.join(
        model_root, f'{set}_R_h264{model_name}.h5'))

    # unify size
    frames = min(len(labels), len(coordinates_L), len(coordinates_R))

    # drop and convert
    X_train_L = np.vstack((X_train_L,
                          coordinates_L.filter(
                              regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]))
    X_train_R = np.vstack((X_train_R,
                          coordinates_L.filter(
                              regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]))
    y_train = np.vstack((y_train, labels[:frames, 6:9]))


from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
X_scaler = MinMaxScaler()
X_train_L = X_scaler.fit_transform(X_train_L)
X_train_R = X_scaler.transform(X_train_R)

y_scaler = MinMaxScaler()
y_train = y_scaler.fit_transform(y_train)



In [ ]:
X_train_L.shape, X_train_R.shape, y_train.shape

# Trainning

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = FCNN(input_dim=X_train_L.shape[1], hidden_dim=64, output_dim=y_train.shape[1], device=device, num_hidden_layers=5)
net.train(X_train_R, X_train_L, y_train, epochs=100, use_tqdm=True, save_loss=True)


# Testing

In [ ]:
# pre setting training sets
test_sets = ['C_M1_T1_2', 'C_M1_T1_4',
             'R2_M1_T1_2', 'R2_M1_T1_4',
             'L2_M1_T1_2', 'L2_M1_T1_4',
             'Z2_M1_T1_2',]

# initial arrays
X_test_L = np.zeros((0, 16))
X_test_R = np.zeros((0, 16))
y_test = np.zeros((0, 3))

for set in tqdm(test_sets):
    # load from files
    labels = np.genfromtxt(os.path.join(
        label_root, set, 'labels_30hz.txt'), delimiter=',')
    coordinates_L = pd.read_hdf(os.path.join(
        model_root, f'{set}_L_h264{model_name}.h5'))
    coordinates_R = pd.read_hdf(os.path.join(
        model_root, f'{set}_R_h264{model_name}.h5'))

    # unify size
    frames = min(len(labels), len(coordinates_L), len(coordinates_R))

    # drop and convert
    X_test_L = np.vstack((X_test_L,
                          coordinates_L.filter(
                              regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]))
    X_test_R = np.vstack((X_test_R,
                          coordinates_L.filter(
                              regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]))
    y_test = np.vstack((y_test, labels[:frames, 6:9]))

X_test_L = X_scaler.transform(X_test_L)
X_test_R = X_scaler.transform(X_test_R)
y_test = y_scaler.transform(y_test)


In [ ]:
X_test_L.shape, X_test_R.shape, y_test.shape

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = net.predict(X_test_R, X_test_L)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error on test data: {mse}')

# Demos

In [ ]:
for i in range(3):
    q1 = np.percentile(y_train[:,i], 25)
    q3 = np.percentile(y_train[:,i], 75)

    print(q1, q3)

In [ ]:
# 绘制直方图
plt.hist(y_train, bins=30, edgecolor='black', label=['x', 'y', 'z'])

# 添加标题和标签
plt.title("Histogram")
plt.xlabel("Value")
plt.ylabel("Frequency")

# 添加图例
plt.legend()

# 显示图形
plt.show()

In [ ]:
# 绘制箱线图
plt.boxplot(y_train, showfliers=False)

# 添加标题和标签
plt.title("Box Plot")
plt.xlabel("Data")

# 显示图形
plt.show()


In [ ]:
import json
import matplotlib.pyplot as plt

# Load the data from the JSON file
with open('params_and_losses*.json', 'r') as f:
    data = json.load(f)

# Create a title with the training parameters
params = {k: v for k, v in data.items() if k != 'losses'}
title = 'Loss over epochs\n' + ', '.join(f'{key}: {value}' for key, value in params.items())

# Separate the epoch and loss values into different arrays
epochs, losses = zip(*data['losses'])

# Plot the loss values
plt.figure(figsize=(10, 6))
plt.plot(epochs, losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title(title)
plt.grid(True)
plt.show()
